# PyOpenCL

## Arrays

`pyopencl.array.Array` provides a numpy-like interface to OpenCL buffers.

In [1]:
import pyopencl as cl
import pyopencl.array
import numpy as np

In [2]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

### Create an array

In [3]:
a_g = cl.array.empty(queue, 10, dtype=np.float32)

Most of numpy's array creation methods supported.

In [26]:
a_g = cl.array.zeros(queue, 10, dtype=np.float32)

In [27]:
b_g = cl.array.zeros_like(a_g)

In [28]:
a_g = cl.array.arange(queue, 0, 10, 1, dtype=np.float32)

### Operations on arrays

`pyopencl.array.Array` works exactly like numpy arrays.
Most of the operations supported by numpy arrays are supported by
`pyopencl.array.Array`.

In [17]:
c_g = a_g ** 2

In [18]:
c_g = a_g + 10.

In [19]:
c_g = 2 * a_g

### Easy data transfer

In [20]:
c_np = c_g.get()
print(c_np)

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


Printing the array directly also supported

In [21]:
print(c_g)

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


To transfer from host to device

In [22]:
c_dev = cl.array.to_device(queue, c_np)

In [23]:
print(c_dev)

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


### The square problem using arrays

In [24]:
a_np = np.arange(0, 10, 1, dtype=np.float32)
print(a_np ** 2)

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]


In [25]:
a_g = cl.array.to_device(queue, a_np)
res_g = a_g ** 2
print(res_g)

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]


### Access OpenCL buffer

Accessing the OpenCL buffer from `Array` is also possible

In [30]:
a_g = cl.array.arange(queue, 0, 10, 1, dtype=np.float32)

In [31]:
prg = cl.Program(ctx, """
    __kernel void square(__global float *a_g)
    {
        int gid = get_global_id(0);
        float a_gid = a_g[gid];
        a_g[gid] = a_gid * a_gid;
    }
""").build()

In [33]:
prg.square(queue, a_g.shape, None, a_g.data)

In [34]:
print(a_g)

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]
